In [1]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from IPython.display import display, Markdown, Latex
import os

os.environ["COHERE_API_KEY"] = "aOWxdujRJfdvlM2YqYwYvjuUbDujqcAFhDYJdnqC"

In [5]:
# save the embeddings in a DB that is persistent
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
loader = TextLoader("data/sources_txt/abc.txt")
document=loader.load()
docs=document[0].page_content
text_splitter = CharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separator ='\n\n',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    add_start_index = True,
)
docs = text_splitter.create_documents([docs])
texts = [doc.page_content for doc in docs]


# Define the embeddings model
embeddings = CohereEmbeddings(model = "embed-multilingual-v2.0")

metadata=[{"source": text} for text in texts]

# Embed the documents and store in index
vector_store = Qdrant.from_texts(texts, embeddings, location=":memory:",metadatas=metadata, collection_name="summaries", distance_func="Dot")

Created a chunk of size 492, which is longer than the specified 100
Created a chunk of size 530, which is longer than the specified 100
Created a chunk of size 471, which is longer than the specified 100
Created a chunk of size 310, which is longer than the specified 100
Created a chunk of size 593, which is longer than the specified 100
Created a chunk of size 475, which is longer than the specified 100
Created a chunk of size 611, which is longer than the specified 100
Created a chunk of size 699, which is longer than the specified 100
Created a chunk of size 390, which is longer than the specified 100
Created a chunk of size 471, which is longer than the specified 100
Created a chunk of size 776, which is longer than the specified 100
Created a chunk of size 593, which is longer than the specified 100
Created a chunk of size 475, which is longer than the specified 100
Created a chunk of size 611, which is longer than the specified 100
Created a chunk of size 699, which is longer tha

In [9]:
retriever=vector_store.as_retriever()

#create custom prompt for your use case
prompt_template="""
Eres un verificador de información. Solo puedes apoyarte en los archivos que tienes como contexto.

En caso de tener información sobre el texto pero esta no forma parte de tu contexto, responde: **No existe información en la base de contexto actual sobre esta afirmación.**

Clasifica las afirmaciones en las siguientes categorías:
- "falso": La información que aparece es completamente falsa acorde al contexto.
- "verdadero": La información que aparece es completamente verdadera acorde al contexto.
- "a medias": La información tiene algunas partes verdaderas y otras falsas acorde al contexto. Puede ocurrir que la información sea verdadera de acuerdo a algunas fuentes y falsa de acuerdo a otras. Si esto ocurre, menciónalo explícitamente.
- "bulo": La información es falsa, escrita desde un punto de vista subjetivo y con intención de desinformar.
- "no hay información": No existe información sobre la temática central de la información en la fuente. Si no existe nada de información al respecto, elige esta opción.

Deberás devolver la salida de la clasificación y además una justifiación de esta clasificación, explicando cuáles de las fuentes y fechas apoyan o refutan tu clasificación.

Debes decir explícitamente qué fuentes apoyan tus elecciones y cuáles no. El nombre de la fuente y la fecha de publicación aparecen antes de cada noticia de los documentos a los que tienes acceso.

El formato de salida, en caso de que la afirmación sea verificable debe ser el siguiente, en Markdown:

**Veredicto**: Falso / Verdadero / A medias / Bulo / No hay información\\

**Justificación**: [Salida de la justificación que produces]\\

**Fuentes**:
- ABC: Falso / Verdadero / A medias / Bulo / No hay información
- ElMundo: Falso / Verdadero / A medias / Bulo / No hay información
- ElPlural: Falso / Verdadero / A medias / Bulo / No hay información
- OKDiario: Falso / Verdadero / A medias / Bulo / No hay información

Cita siempre las cuatro fuentes. En caso de no haber información en dicha fuente, dilo explícitamente. Asegúrate siempre de que una afirmación es 100'%' exacta para clasificarla como verdadera.

Para verificar la afirmación "El precio de la vivienda en Barcelona." tienes que consultar las fuentes que tienes como contexto. Si según ABC y ElMundo es verdadero, según OKDiario es falso y no hay información en ElPlural, la clasificación será "A medias". Si no hay información en ninguna de las fuentes, la respuesta será "No hay información".

Esto es muy importante para mi trabajo. Recuerda decir que no hay información en caso de que no tengas conocimiento en el contexto, no inventes absolutamente nada.

Si no tienes información en el respecto sobre la afirmación, di:**No existe información en la base de contexto actual sobre esta afirmación.**

Tu único conocimiento disponible es tu contexto.
----------------
{summaries}"""

messages = [
    SystemMessagePromptTemplate.from_template(prompt_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

chain_type_kwargs = {"prompt": prompt}

llm=Cohere(model="command")

#build your chain for RAG+C
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

#print your results with Markup language
def print_result(result):
  output_text = f"""### Question: 
  {query}
  ### Answer: 
  {result['answer']}
  ### Sources: 
  {result['sources']}
  ### All relevant sources:
  {' '.join(list(set([doc.metadata['source'] for doc in result['source_documents']])))}
  """
  return(output_text)

In [11]:
result

{'question': 'El paro ha bajado',
 'answer': ' Verdadero: La tasa de temporalidad bajó en 2023 hasta el 16,5% tras aumentar los asalariados con contrato indefinido en más de 800.000 (+5,6%) y reducirse los contratados de manera temporal en 140.300 (-4,5%). \n\nFuentes: La tasa de temporalidad bajó en 2023 hasta el 16,5% tras aumentar los asalariados con contrato indefinido en más de 800.000 (+5,6%) y reducirse los contratados de manera temporal en 140.300 (-4,5%).  La caída es sustancialmente menor a la pérdida de ocupados que se produjo en el pasado ejercicio, cuando se perdieron 81.900 empleos. \n\nLa afirmación es verdadera sobre la base de los datos proveidos por el comentario. ',
 'sources': '',
 'source_documents': [Document(page_content='Por su parte, el número de parados disminuyó en el último trimestre en 24.600 personas, hasta 2.830.600 desempleados. Si bien, en este caso la tendencia se presenta más similar a la experimentada en años anteriores en este periodo. De hecho, la 

In [12]:
query = "Yolanda Díaz vive en el ministerio"
result = chain(query)
display(Markdown(print_result(result)))

### Question: 
  Yolanda Díaz vive en el ministerio
  ### Answer: 
   La afirmación **"Yolanda Díaz vive en el ministerio"** no existe en la base de contexto actual sobre esta afirmación, por lo que la respuesta final sería: 

**Veredicto**: No existe información en la base de contexto actual sobre esta afirmación.

**Justificación**: [La afirmación no está verificable dado que no existe en ninguna de las fuentes a las que tiene acceso el asistente.]

**Fuentes**:
- ABC:
- ElMundo:
- ElPlural:
- OKDiario:

Si desea, puede emplear el código del Markdown para agregar más información a la salida de verificación. 
  ### Sources: 
  
  ### All relevant sources:
  NOTICIAS RELACIONADAS
Menos de un 10% de los casi tres millones de parados en España realiza cursos de formación
Menos de un 10% de los casi tres millones de parados en España realiza cursos de formación
GONZALO D. VELARDE
Los sectores a los que más afectará la subida del SMI: más de un 30% de sus trabajadores cobran el mínimo legal
Los sectores a los que más afectará la subida del SMI: más de un 30% de sus trabajadores cobran el mínimo legal
GONZALO D. VELARDE
Incluso estas cifras vienen en cierto modo maquilladas por la ganancia de empleo en el sector público, que permitió aminorar el ajuste de ocupación. Mientras que el empleo privado disminuyó este trimestre en 77.600 personas, hasta 17.653.600 trabajadores, el empleo público aumentó en 58.600, hasta 3.593.300. Incluso estas cifras vienen en cierto modo maquilladas por la ganancia de empleo en el sector público, que permitió aminorar el ajuste de ocupación. Mientras que el empleo privado disminuyó este trimestre en 77.600 personas, hasta 17.653.600 trabajadores, el empleo público aumentó en 58.600, hasta 3.593.300. La tasa de temporalidad bajó en 2023 hasta el 16,5% tras aumentar los asalariados con contrato indefinido en más de 800.000 (+5,6%) y reducirse los contratados de manera temporal en 140.300 (-4,5%). ﻿ El empleo resiste con 783.000 ocupados más y una caída de casi 200.000 parados pese al mal cierre de 2023
El mercado laboral perdió 19.000 trabajadores en el cuarto trimestre del pasado año mientras el desempleo disminuyó en 24.600 personas
El total de ocupados se mantiene por encima de los 21,2 millones mientras el paro alcanza a 2,8 millones de españoles, situándose la tasa en el 11,7%
El recorte de jornada laboral que planea Díaz liquidará un tercio de la creación de empleo para 2024
  